# DATA101 Final Project: Interactive Visualization Application
## Part 2: Non-Spatial Data Visualization

## Import Libraries

In [6]:
# Install Plotly
# Using pip
!pip install plotly

# Using conda
# conda install -c conda-forge plotly

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
## Only do this if you're working on Google Colab

# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [8]:
# when using Google Colab
# dataset_folder = Path('/gdrive/MyDrive/datasets')

# when using local folder
dataset_folder = Path('cleaned-datasets/')

## Datasets

In [13]:
# Primary Completion Rates
primary_completion = pd.read_csv(dataset_folder / 'Primary/' 'Primary_Completion_Rate_by_Region_and_Year.csv', index_col=None)

# Primary Drop-out Rates
primary_dropout = pd.read_csv(dataset_folder / 'Primary/' 'Primary_Drop-out_Rate_by_Region_and_Year.csv', index_col=None)

# Primary Net Enrollment Rates
primary_enrollment = pd.read_csv(dataset_folder / 'Primary/' 'Primary_Net_Enrollment_Rate_by_Region_and_Year.csv', index_col=None)

# Secondary Completion Rates
secondary_completion = pd.read_csv(dataset_folder / 'Secondary/' 'Secondary_Completion_Rate_by_Region_and_Year.csv', index_col=None)

# Secondary Drop-out Rates
secondary_dropout = pd.read_csv(dataset_folder / 'Secondary/' 'Secondary_Drop-out_Rate_by_Region_and_Year.csv', index_col=None)

# Secondary Net Enrollment Rates
secondary_enrollment = pd.read_csv(dataset_folder / 'Secondary/' 'Secondary_Enrollment_Rate_by_Region_and_Year.csv', index_col=None)

# Poverty Incidence Rates
poverty_incidence = pd.read_csv(dataset_folder / 'Poverty_Incidence_among_Population.csv', index_col=None)

In [14]:
primary_completion

,Region,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,NCR,88.5,85.3,85.3,79.7,82.9,83.1,80.0,78.7,74.7,82.3
1,CAR,75.0,71.7,75.8,77.1,71.8,78.9,78.2,82.6,86.4,86.2
2,Region I,81.6,82.7,84.8,83.1,82.8,82.4,85.2,91.1,94.5,91.5
3,Region II,78.5,78.1,80.4,78.3,77.5,73.1,82.0,87.9,87.1,89.0
4,Region III,82.2,80.7,81.4,81.9,80.2,80.3,82.6,90.8,88.9,89.1
5,Region IV-A,81.8,79.3,81.6,84.8,82.1,85.4,84.8,72.5,92.3,88.5
6,Region IV-B,70.0,68.1,71.6,69.4,72.9,72.4,74.2,78.1,86.9,90.2
7,Region V,73.0,74.4,74.4,74.4,74.5,74.8,76.8,85.0,86.3,85.4
8,Region VI,70.6,71.9,72.6,72.5,71.2,74.6,78.6,85.7,90.1,88.9
9,Region VII,77.8,78.4,78.8,76.8,76.6,80.4,80.9,85.8,86.1,88.5


## Bar Chart

The bar chart is defined by the following filters:
- Region (Multi-select) <- horizontal if >2, vertical if 1
- Education Level (Primary or Secondary? Single-select)
- Education Metric (Enrollments, Completions, or Drop-outs? Single-select) <- this defines the top5 and widest bar to be featured

Interactivity:
??? Year range slider

## Line Chart

The line chart is defined by the following filters:

- Region (Multi-select) <- affects the legend, 1 color per region
- Education Level (Primary or Secondary? Single-select)
- Education Metric (Enrollments, Completions, or Drop-outs? Single-select) <- this defines the data to be used

Interactivity:
- Year Range Slider <- this identifies the x-axis values to be included

## Scatterplot

The scatterplot is defined by the following filters:

- Region (Multi-select) <- affects the legend, 1 color per region
- Education Level (Primary or Secondary? Single-select)
- Education Metric (Enrollments, Completions, or Drop-outs? Single-select) <- this defines the data to be used

Interactivity:
- Year Range Slider <- this identifies the x-axis values to be included

TO FIX!